In [25]:
import numpy.matlib 
import numpy as np 

def getxy(nlist, x,y):
    for n in nlist:
        if n.x == x and n.y == y:
            return n
    return False

class node:
    def __init__(self,name,data,x=0, y=0):
        self.node_data = [list(x) for x in data]
        self.node_name = name
        self.x = x
        self.y = y
        self.static = False  # is this locked in place
        self.attached = [False,False,False,False] # who is attached to the top,right,bottom,left
        
    def setStatic(self,v):
        self.static = v
        
    def __str__(self):
        return "\n".join( [ "".join(x) for x in self.node_data ] )
        
    # 0 = top
    # 1 = right
    # 2 = bottom
    # 3 = left
    def get_edge(self,n):
        if n == 0:
            return "".join(self.node_data[0])
        elif n == 1:
            return "".join([ x[-1] for x in self.node_data ])
        elif n == 2:
            return "".join(self.node_data[-1])
        elif n == 3:
            return "".join([ x[0] for x in self.node_data ])

    def flip(self):
        if self.static:
            #print("error {} is static".format( self.node_name))
            return False
        a = np.matrix( self.node_data )
        b = np.fliplr( a)
        self.node_data = b.tolist()
        return True
    
    # https://stackoverflow.com/questions/42519/how-do-you-rotate-a-two-dimensional-array
    def rotate_cw_90(self):
        if self.static:
            #print("error {} is static".format( self.node_name))
            return False
        a = np.matrix( self.node_data )
        b = np.fliplr( a.transpose() )
        self.node_data = b.tolist()
        return True
            

def flipString(s):
    a = np.matrix([list(x) for x in s.strip().split('\n')])
    b = np.fliplr(a).tolist()
    return "\n".join(["".join(x) for x in b])

def rotateStringcw90(s):
    a = np.matrix([list(x) for x in s.strip().split('\n')])
    b = np.fliplr( a.transpose() ).tolist()

    return "\n".join(["".join(x) for x in b])



In [57]:
import re

input_file="data/day20_sample_1.txt"
#input_file="data/day20.txt"
l1 = [ x.strip() for x in open(input_file).readlines()]


## Read in the tiles
tile=""
tiles = {}
connections = {}
for line in l1:
    if line == "":
        continue
    if line.find("Tile") != -1:
        m = re.match("Tile (\d+):", line)
        tile = m[1]
        connections[tile] = []
    else:
        if tile in tiles:
            tiles[tile].append(line)
        else:
            tiles[tile] = [line]

nodes = []            
for t in tiles:
    nodes.append( node(t, tiles[t]))
 
work_q = []
finished = []

work_q.append( nodes[0])
nodes[0].x = 0
nodes[0].y = 0
nodes[0].setStatic(True)

while len(work_q) > 0:
    current_node = work_q.pop(0)
    #print("processing {}".format(current_node.node_name))
    for side in [0,1,2,3]: #top, right, bottom, left
        for check_node in nodes:
            if check_node == current_node:
                continue  # don't check yourself
            mate_side = [2,3,0,1][side]  # find the mating side
            xmove     = [0,1,0,-1][side] # which way to move the x
            ymove     = [1,0, -1,0][side] # which way to move the y
            for flip in [0,1]: # try it flipped and unfliped
                for rot in [0,1,2,3]: # try every rotation
                    if current_node.get_edge(side) == check_node.get_edge(mate_side):
                        if check_node not in work_q and check_node not in finished:
                            work_q.append(check_node)
                        current_node.attached[side] = check_node.node_data
                        check_node.x = current_node.x + xmove
                        check_node.y = current_node.y + ymove
                        check_node.setStatic(True)
                    check_node.rotate_cw_90()
                check_node.flip()
    finished.append(current_node)

for idx,n in enumerate(nodes):
    print(idx, n.node_name)

## Upper left corner to start with
xmin=1000
ymin=1000
xmax=0
ymax=0
for n in nodes:
    if n.x < xmin:
        xmin = n.x
    if n.y < ymin:
        ymin = n.y
    if n.x > xmax:
        xmax = n.x
    if n.y > ymax:
        ymax = n.y
print(xmin,ymin, xmax,ymax)


print("--")
puz_map= ""
for y in range(ymin, ymax+1):
    w = []
    for x in range(xmin, xmax+1):
        n = getxy(nodes,x,y)
        print(n.node_name,x,y)
        w.append(n)
    for yy in range(1,9):
        puz_map += ( "".join([ "".join(j.node_data[yy][1:-1]) for j in w ]  ))
        puz_map += "\n"
    #a+="---\n"


print(puz_map)

0 2311
1 1951
2 1171
3 1427
4 1489
5 2473
6 2971
7 2729
8 3079
-1 0 1 2
--
1951 -1 0
2311 0 0
3079 1 0
2729 -1 1
1427 0 1
2473 1 1
2971 -1 2
1489 0 2
1171 1 2
.####...#..#.....#......
....#..#...##..#.#.###..
...########.#....#####.#
##.#....#.##.####...#.##
###.#####...#.#####.#..#
##.##.###.#.#..######...
###....#.#....#..#......
.#.#..#.##...#.##..#####
###.#...#..#.##.######..
.#.#....#.##.#...###.##.
...#..#..#.#.##..###.###
##..##.#...#...#.#.#.#..
#.####....##..########.#
###.#.#...#.######.#..##
#.####..#.####.#.#.###..
#..#.##..#..###.#.##....
...###...##...#...#..###
.#.###..##..##..####.##.
#.##..#..#...#..####...#
#####..#####...###....##
#..####...#.#.#.###.###.
..#.#..#..#.#.#.####.###
.####.###.#...###.#..#.#
.#.#.###.##.##.#..#.##..



In [53]:
regex1="#....##....##....###"
regex2="#..#..#..#..#..#"

re1=".####...#####..#...###.."

p1 = flipString(puz_map)
map1 = rotateStringcw90(puz_map)
map2 = rotateStringcw90(map1)
map3 = rotateStringcw90(map2)
map4 = rotateStringcw90(map3)


print(  re.findall(re1, puz_map) )

print(map3)

[]
..###..#..#.##..#.##.##.
...#...#.##..#..##.###..
.##....######.#.##..####
.#####.##...#.#.....#..#
.##..#.#########.##..#..
..#.##..###.#..####.####
###.###.#######..#####..
...###.##...##.#.######.
.#.###.#...####....#..##
...##.#.#.#.##.##...###.
...#...####..###.##....#
.##.##.......###.#..##.#
##.#....#####.#....#....
..##.#...#..###.#..#####
..#...##..#....#####...#
#.####.###....#..#.#..#.
.##.###....#.......#.###
..#.##.#..#..#.#..#.#.##
..#.#......##.####..#..#
###.##..#..####.##.####.
#.##.#.#.##.#.#######.##
#...#.#.#...###...##.##.
#..#######.#.#...#.#..##
...####.#..#####..###...


1951    2311    3079
2729    1427    2473
2971    1489    1171

In [27]:


print(rotateStringcw90(puz_map) )
#a = np.matrix([list(x)for x in puz_map.split("\n")])
#b = np.fliplr( a.transpose() ).tolist()


#[x for x in b]
#a

...###..#####..#.####...
##..#.#...#.#.#######..#
.##.##...###...#.#.#...#
##.#######.#.##.#.#.##.#
.####.##.####..#..##.###
#..#..####.##......#.#..
##.#.#..#.#..#..#.##.#..
###.#.......#....###.##.
.#..#.#..#....###.####.#
#...#####....#..##...#..
#####..#.###..#...#.##..
....#....#.#####....#.##
#.##..#.###.......##.##.
#....##.###..####...#...
.###...##.##.#.#.#.##...
##..#....####...#.###.#.
.######.#.##...##.###...
..#####..#######.###.###
####.####..#.###..##.#..
..#..##.#########.#..##.
#..#.....#.#...##.#####.
####..##.#.######....##.
..###.##..#..##.#...#...
.##.##.#..##.#..#..###..


In [17]:
nodes[0].node_data

[['.', '.', '#', '#', '.', '#', '.', '.', '#', '.'],
 ['#', '#', '.', '.', '#', '.', '.', '.', '.', '.'],
 ['#', '.', '.', '.', '#', '#', '.', '.', '#', '.'],
 ['#', '#', '#', '#', '.', '#', '.', '.', '.', '#'],
 ['#', '#', '.', '#', '#', '.', '#', '#', '#', '.'],
 ['#', '#', '.', '.', '.', '#', '.', '#', '#', '#'],
 ['.', '#', '.', '#', '.', '#', '.', '.', '#', '#'],
 ['.', '.', '#', '.', '.', '.', '.', '#', '.', '.'],
 ['#', '#', '#', '.', '.', '.', '#', '.', '#', '.'],
 ['.', '.', '#', '#', '#', '.', '.', '#', '#', '#']]

In [24]:
np.matrix([list(x) for x in puz_map.strip().split("\n")] )

matrix([['.', '#', '#', '#', '#', '.', '.', '.', '#', '.', '.', '#', '.',
         '.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '#', '.', '.', '#', '.', '.', '.', '#', '#',
         '.', '.', '#', '.', '#', '.', '#', '#', '#', '.', '.'],
        ['.', '.', '.', '#', '#', '#', '#', '#', '#', '#', '#', '.', '#',
         '.', '.', '.', '.', '#', '#', '#', '#', '#', '.', '#'],
        ['#', '#', '.', '#', '.', '.', '.', '.', '#', '.', '#', '#', '.',
         '#', '#', '#', '#', '.', '.', '.', '#', '.', '#', '#'],
        ['#', '#', '#', '.', '#', '#', '#', '#', '#', '.', '.', '.', '#',
         '.', '#', '#', '#', '#', '#', '.', '#', '.', '.', '#'],
        ['#', '#', '.', '#', '#', '.', '#', '#', '#', '.', '#', '.', '#',
         '.', '.', '#', '#', '#', '#', '#', '#', '.', '.', '.'],
        ['#', '#', '#', '.', '.', '.', '.', '#', '.', '#', '.', '.', '.',
         '.', '#', '.', '.', '#', '.', '.', '.', '.', '.', '.'],
        ['.', '#', '.', '#'

In [ ]:
a = "#.##...##."
a[1:-1]

In [55]:
nodes[1].node_data, nodes[1].node_name

([['#', '.', '#', '#', '.', '.', '.', '#', '#', '.'],
  ['#', '.', '#', '#', '#', '#', '.', '.', '.', '#'],
  ['.', '.', '.', '.', '.', '#', '.', '.', '#', '#'],
  ['#', '.', '.', '.', '#', '#', '#', '#', '#', '#'],
  ['.', '#', '#', '.', '#', '.', '.', '.', '.', '#'],
  ['.', '#', '#', '#', '.', '#', '#', '#', '#', '#'],
  ['#', '#', '#', '.', '#', '#', '.', '#', '#', '.'],
  ['.', '#', '#', '#', '.', '.', '.', '.', '#', '.'],
  ['.', '.', '#', '.', '#', '.', '.', '#', '.', '#'],
  ['#', '.', '.', '.', '#', '#', '.', '#', '.', '.']],
 '1951')